In [1]:
!pip install requests                 #Requesting the URL
!pip install bs4                      #Web Scrapping 
!pip install validator_collection     #Validator the conditions 
!pip install git+"https://github.com/codelucas/newspaper.git"
!pip install goose3 #Webwrapper Package
!pip install nltk
!pip install sumy
!pip install easy-rouge

  Cloning https://github.com/codelucas/newspaper.git to /tmp/pip-req-build-57bt2y4i
  Running command git clone -q https://github.com/codelucas/newspaper.git /tmp/pip-req-build-57bt2y4i
     |████████████████████████████████| 194kB 4.2MB/s 
     |████████████████████████████████| 7.4MB 56.6MB/s 
     |████████████████████████████████| 11.1MB 36.8MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
  Created wheel for newspaper3k: filename=newspaper3k-0.2.8-cp36-none-any.whl size=214639 sha256=36b967f7cafde6897704e91196ea128881c46c5c485789153935f8db312d9258
  Stored in directory: /tmp/pip-ephem-wheel-cache-2mgpm2sr/wheels/9c/bf/af/bb27f46a20721b29a9623b572b89928ebe96f255f398e63fa4
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=88b29c083308a7e847785498c4ce9ec496a3a05e9b3f7bf4e2cc7d0661290ca0
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for feedparser: filen

In [0]:
from bs4 import BeautifulSoup
import re
import requests
from validator_collection import checkers
import dateutil.parser
from datetime import date

# Pages from which the newsletters are scrapped
## Update the pages on a weekly basis for weekly updates
Pages=["https://awsinsider.net/Home.aspx","https://dataengweekly.substack.com/",
       "https://news.dataelixir.com/t/t-EBBA06B24152E04B2540EF23F30FEDED",
       "https://www.datascienceweekly.org/newsletters/data-science-weekly-newsletter-issue-315"]

#Sources of the newsletters
Sources=["awsinsider","dataengweekly",
         "dataelixir","datascienceweekly"]

#Pages=["https://awsinsider.net/Home.aspx",
#       "https://www.datascienceweekly.org/newsletters/data-science-weekly-newsletter-issue-315"]

#Sources of the newsletters
#Sources=["awsinsider","datascienceweekly"]


#Request 
links=[]
#Looping the differenent webpages
for i in range(0,len(Pages)):
  # Sending request to each page
   res = requests.get(Pages[i]) 
  # Taking all text value from the webpage  
   Soup=BeautifulSoup(res.text,'lxml')
   #Inside the webpage we have several other links
   NewsLinks=[]
   #Finding all the a tag in the webpage
   for a in Soup.find_all('a', href=True):
    #Checking is this a webpage url or not 
    if checkers.is_url(a['href']):
      # if Webpage url is below vales then append
      if Sources[i]=="awsinsider" and "articles" in a['href'] and a['href'] not in NewsLinks and re.search('\d{4}/\d{2}/\d{2}|\d{4}/\w{3}/\d{2}|\d{2}/\d{2}/\d{4}|\d{2}/\w{3}/\d{4}', a['href']):
        NewsLinks.append(a['href'])
      elif Sources[i]=="dataengweekly" and a['href'] not in NewsLinks:
        NewsLinks.append(a['href'])
      elif Sources[i]=="dataengweekly" and "http://www.datadog.com" in a['href'] :
        break
      elif Sources[i]=="dataelixir" and "forwardtomyfriend" in a['href']:
        break
      elif Sources[i]=="dataelixir": #and "news.dataelixir" not in a['href']:
        NewsLinks.append(a['href'])
      elif Sources[i]=="dataelixir" and "threader.app" not in a['href']:
        NewsLinks.append(a['href'])
      elif Sources[i]=="datascienceweekly" and "dataform" in a["href"]:
        break
      elif Sources[i]=="datascienceweekly" and "thisismetis" not in a['href']:
        NewsLinks.append(a['href'])
      elif Sources[i]=="datascienceweekly" and "threader.app" not in a['href']:
        NewsLinks.append(a['href'])
   links.append(NewsLinks)

In [3]:
len(links)

4

In [4]:
from newspaper import Article
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.summarization.summarizer import summarize
import datetime
import pytz
utc=pytz.UTC
from datetime import timezone
from goose3 import Goose

summary=[]
link=[]
title=[]
keywords=[]
date=[]
Source=[]
Group=[]
Text=[]
count=1
g = Goose()
for page in links:
  summary_=[]
  link_=[]
  title_=[]
  keywords_=[]
  date_=[]
  Source_=[]
  text_=[]
  #print(page)
  for url in page:
    try:
      #article = g.extract(url)
      #For different language newspaper refer above table 
      toi_article = Article(url, language="en") # en for English 
    
      #To download the article 
      toi_article.download() 
  
      #To parse the article 
      toi_article.parse() 
    
      #To perform natural language processing ie..nlp 
      toi_article.nlp() 
    
      if len(toi_article.summary.splitlines())>=5 :#and #(toi_article.publish_date)>= (datetime.datetime.now() - datetime.timedelta(-7)):
        if len(word_tokenize(toi_article.summary))>60:
          summary_.append(summarize(toi_article.text.encode('utf8').decode('ascii', 'ignore'),word_count =60))
        else:
          summary_.append(toi_article.summary)
        link_.append(url)
        if toi_article.title!="Quanta Magazine":
          title_.append(toi_article.title)
        else:
          article = g.extract(url)
          title_.append((article.title))
        keywords_.append(toi_article.keywords)
        date_.append(toi_article.publish_date)
        text_.append(toi_article.text)
        if count <= 2:
          Source_.append("Data Engineering")
        else:
          Source_.append("Data Science")

    except:
      continue
      
    #try:|
    #  
    #except:
    #  title_.append((toi_article.title))
  count=count+1
  summary.append(summary_)
  link.append(link_)
  title.append(title_)
  keywords.append(keywords_)
  date.append(date_)
  Source.append(Source_)
  Text.append(text_)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


def Data_cleaning(text):
    
    text=text.lower()
    
    text=re.sub("<!--?.*?-->","",text)
    #text=re.sub("\u","",text)
    text=re.sub("(\\d|\\W)+"," ",text)

    #text=re.sub('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'," ",text)

    stop_words=set(stopwords.words('english'))

    words=word_tokenize(text)

    punctuated_sentence=[]

    for i in words:
        if i not in string.punctuation:
            punctuated_sentence.append(i)
          
    Preprocessed_data=""
    for i in (punctuated_sentence):
      Preprocessed_data=Preprocessed_data+" "+i
      
    return Preprocessed_data
  
cleaned_blog=[]  
for blog in ((Text)):
  cleaned_article=[]
  for article in ((blog)):
    cleaned_article.append(Data_cleaning(article))
  cleaned_blog.append(cleaned_article)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
cleaned_blog[1]

[' this week s issue has posts from scribd and slack on apache airflow using envoy with apache kafka the open sourcing of linkedin s datahub elasticsearch in production and apache flink s new sql ddl support also a post on the data infrastructure behind the spotify wrapped campaign and an article with advice on running a data team slack wrote about their experiences upgrading apache airflow from version which they had been running for two years to version the post describes the upgrade strategies that they considered the steps they took many around schema changes and backing up the metadata database how they tested the upgrade and some issues they found after the upgrade https slack engineering reliably upgrading apache airflow at slacks scale a f d a spotify writes about their large scale analysis of a decade of playback data to power their spotify wrapped campaign at the end of they performed a number of intermediate jobs which allowed them to more quickly iterate and verify the qual

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
Score_blog=[]
index=[]
for blog in cleaned_blog:
  t=[]
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform(blog)
  feature_names = vectorizer.get_feature_names()
  dense = vectors.todense()
  denselist = dense.tolist()
  df = pd.DataFrame(denselist, columns=feature_names)
  df['mean'] = df.mean(axis=1)
  Score_blog.append(max(df['mean']))
  index.append(np.argsort(list(df['mean']))[-2:])

In [8]:
links[1]

['https://support.substack.com/hc/en-us?s=support%2Bdataengweekly%40substack.com',
 'https://dataengweekly.substack.com/p/data-eng-weekly-356',
 'https://slack.engineering/reliably-upgrading-apache-airflow-at-slacks-scale-2a31f3d03a06',
 'https://labs.spotify.com/2020/02/18/wrapping-up-the-decade-a-data-story/',
 'https://tech.scribd.com/blog/2020/modernizing-an-old-data-pipeline.html',
 'https://about.gitlab.com/blog/2020/02/10/lessons-learned-as-data-team-manager/',
 'https://medium.com/@adam.kotwasinski/deploying-envoy-and-kafka-8aa7513ec0a0',
 'https://engineering.linkedin.com/blog/2020/open-sourcing-datahub--linkedins-metadata-search-and-discovery-p',
 'https://flink.apache.org/news/2020/02/20/ddl.html',
 'https://blog.acolyer.org/2020/02/21/extending-relational-query-processing/',
 'https://facinating.tech/2020/02/22/in-depth-guide-to-running-elasticsearch-in-production/',
 'https://substack.com/about?utm_source=substack&utm_medium=web&utm_content=footer',
 'https://enable-javasc

In [9]:
index

[array([1, 7]), array([9, 4]), array([2, 0]), array([4, 1])]

In [0]:
Selected_Text=[]
Selected_link=[]
Selected_title=[]
Selected_keywords=[]
Selected_date=[]
Selected_Source=[]
Selected_Summary=[]
c=0
for i in ((index)):
  for j in i: 
    Selected_Text.append(Text[c][j])
    Selected_link.append(link[c][j])
    Selected_title.append(title[c][j])
    Selected_keywords.append(keywords[c][j])
    Selected_date.append(date[c][j])
    Selected_Source.append(Source[c][j])
#    Selected_Summary.append(summary[c][j])
  c=c+1

In [0]:
def Kmeans_summary(text_):

  from sklearn.feature_extraction import text
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.cluster import KMeans

  punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
  stop_words = text.ENGLISH_STOP_WORDS.union(punc)

  Kmeans_summary=[]
  import numpy as np
  from sklearn.metrics import pairwise_distances_argmin_min
  from nltk.tokenize import sent_tokenize
  sentences=sent_tokenize(text_)
  vectorizer = TfidfVectorizer(stop_words = stop_words)
  X = vectorizer.fit_transform(sentences)
  n_clusters=int(np.floor(len(sentences)**0.5))
  print(n_clusters)
  kmeans = KMeans(n_clusters=n_clusters,init='k-means++',max_iter=300,n_init=10,random_state=0)
  kmeans.fit(X)
  avg = []
  for j in range(1):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
  closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,X)
  ordering = sorted(range(1), key=lambda k: avg[k])
  Kmeans_summary=('.'.join([sentences[closest[idx]] for idx in ordering]))  

  return Kmeans_summary

In [0]:
def Textrank_Summary(text):
  from gensim.summarization import summarize
  text= '\n'.join(text.split("."))
  summary=(summarize(text))

  return summary

In [13]:
import nltk
nltk.download('punkt')
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
def LSA_Summary(url):
  

  from sumy.parsers.html import HtmlParser
  from sumy.parsers.plaintext import PlaintextParser
  from sumy.nlp.tokenizers import Tokenizer
  from sumy.summarizers.lsa import LsaSummarizer as Summarizer
  from sumy.nlp.stemmers import Stemmer
  from sumy.utils import get_stop_words


  LANGUAGE = "english"
  SENTENCES_COUNT = 4

  parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    
  stemmer = Stemmer(LANGUAGE)

  summarizer = Summarizer(stemmer)
  summarizer.stop_words = get_stop_words(LANGUAGE)
  s=""
  for sentence in summarizer(parser.document, SENTENCES_COUNT):
    s=s+str(sentence)
  Lsa=s
    
  return Lsa

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def Lexrank_Summary(url):
  from sumy.parsers.html import HtmlParser
  from sumy.parsers.plaintext import PlaintextParser
  from sumy.nlp.tokenizers import Tokenizer
  from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
  from sumy.nlp.stemmers import Stemmer
  from sumy.utils import get_stop_words


  LANGUAGE = "english"
  SENTENCES_COUNT = 10

  parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
  # or for plain text files
  # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
  stemmer = Stemmer(LANGUAGE)

  summarizer = Summarizer(stemmer)
  summarizer.stop_words = get_stop_words(LANGUAGE)

  s=""
  for sentence in summarizer(parser.document, SENTENCES_COUNT):
    s=s+str(sentence)
  Lex=s
  
  return Lex

In [0]:
def Reduction_Summary(url):
  from sumy.parsers.html import HtmlParser
  from sumy.parsers.plaintext import PlaintextParser
  from sumy.nlp.tokenizers import Tokenizer
  from sumy.summarizers.reduction import ReductionSummarizer as Summarizer
  from sumy.nlp.stemmers import Stemmer
  from sumy.utils import get_stop_words


  LANGUAGE = "english"
  SENTENCES_COUNT = 10

  parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
  # or for plain text files
  # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
  stemmer = Stemmer(LANGUAGE)
  summarizer = Summarizer(stemmer)
  summarizer.stop_words = get_stop_words(LANGUAGE)

  s=""
  for sentence in summarizer(parser.document, SENTENCES_COUNT):
     s=s+str(sentence)
  reduction=s
    
  return reduction

In [0]:
def Luhn_summary(url):
  from sumy.parsers.html import HtmlParser
  from sumy.parsers.plaintext import PlaintextParser
  from sumy.nlp.tokenizers import Tokenizer
  from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
  from sumy.nlp.stemmers import Stemmer
  from sumy.utils import get_stop_words


  LANGUAGE = "english"
  SENTENCES_COUNT = 10


  parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
  # or for plain text files
  # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
  stemmer = Stemmer(LANGUAGE)

  summarizer = Summarizer(stemmer)
  summarizer.stop_words = get_stop_words(LANGUAGE)
  s=""
  for sentence in summarizer(parser.document, SENTENCES_COUNT):
    s=s+str(sentence)
  Luhn=(s)
  return Luhn 

In [0]:
def Summary_generation(text,url):

  #Generating Individual Summaries
  Summary=[]
  Summary.append(Kmeans_summary(text))
  Summary.append(Textrank_Summary(text))
  Summary.append(LSA_Summary(url))
  Summary.append(Lexrank_Summary(url))
  Summary.append(Reduction_Summary(url))
  Summary.append(Luhn_summary(url))

  Scores=[]
  #Finding the Scores of The individual Summary
  from rouge.rouge import rouge_n_sentence_level
  target=Summary[1]
  for i in range(0,len(Summary)):
      recall, precision, rouge = rouge_n_sentence_level(Summary[i],target, 2)
      Scores.append(rouge)
  Key=sorted(range(len(Scores)), key=lambda i: Scores[i])[-3:]
  
  #Stacking the top three models
  from gensim.summarization.summarizer import summarize
  cols=['Kmeans F-measure', 'TextRank F-measure', 'LSA F-measure','Lex Rank F-measure', 'Reduction F-measure', 'Luhn F-measure']
  s=""
  for i in range(0,len(Key)):
    s=s+""+Summary[Key[i]]
  
  #Final Summary
  from gensim.summarization.summarizer import summarize
  txt= '\n'.join(s.split("."))
  txt=summarize(txt,word_count=40)
  txt=txt.split("\n")
  sentence=[]
  for i in txt:
    if i not in sentence:
      sentence.append(i)
    sent='.'.join(sentence)
  return sent   

In [18]:
Selected_Summary=[]
for i in range(0,len(Selected_Text)):
  Selected_Summary.append(Summary_generation(Selected_Text[i],Selected_link[i]))

4


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(112 unique tokens: ['john', 'backup', 'nw', 'recoveri', 'water']...) from 22 documents (total 314 corpus positions)
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(239 unique tokens: ['new', 'aw', 'broad', 'dev', 'edg']...) from 44 documents (total 526 corpus positions)


6


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(133 unique tokens: ['amazon', 'artifici', 'aw', 'base', 'big']...) from 28 documents (total 434 corpus positions)


13


...

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(424 unique tokens: ['answer', 'bryjngjolffson', 'coher', 'erik', 'expert']...) from 44 documents (total 958 corpus positions)
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(621 unique tokens: ['artifici', 'chollet', 'françoi', 'gener', 'googl']...) from 154 documents (total 1476 corpus positions)


12


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(307 unique tokens: ['abstract', 'achiev', 'arc', 'call', 'corpu']...) from 49 documents (total 795 corpus positions)
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(632 unique tokens: ['car', 'curiou', 'cybertruck', 'design', 'histori']...) from 154 documents (total 1232 corpus positions)


11


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(287 unique tokens: ['calcul', 'coordin', 'current', 'data', 'databa']...) from 49 documents (total 722 corpus positions)


In [0]:
import pandas as pd
NewsLetter = pd.DataFrame(
    {'Title':Selected_title,
      'Link':Selected_link,
      'Summary':Selected_Summary,
      'Date':Selected_date,
      'Keywords': Selected_keywords,
      'Source':  Selected_Source
    })


In [0]:
for i in range(0,len(NewsLetter)):
  if NewsLetter.Summary[i]=="":
    toi_article = Article(NewsLetter.Link[i], language="en") # en for English 
    #To download the article 
    toi_article.download() 
    #To parse the article 
    toi_article.parse() 
    
    #To perform natural language processing ie..nlp 
    toi_article.nlp()

    NewsLetter.Summary[i]=toi_article.summary

In [0]:
t=NewsLetter.Summary[4].split("\n")
del t[0]
NewsLetter.Summary[4]=".".join(t)

In [22]:
for i in range(0,len(NewsLetter.Summary)):
  temp=NewsLetter.Summary[i].split("\n")
  for j in range(0,len(temp)):
    print()
    if NewsLetter.Title[i]==temp[j]:
      temp.remove(temp[j])
      print(temp[j])
  NewsLetter.Summary[i]=".".join(temp)

In [0]:
from google.colab import files

NewsLetter.to_csv('NewsLetter.csv')
files.download('NewsLetter.csv')

In [0]:
import smtplib
import email.message
from jinja2 import Template
server = smtplib.SMTP('smtp.gmail.com:587')

email_content =""" <html>
 
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    
   <title>Newsletter of Newsletters</title>
   <style type="text/css">
    a {color: #d80a3e;}
  body, #header h1, #header h2, p {margin: 0; padding: 0;}
  #main {border: 1px solid #cfcece;}
  img {display: block;}
  #top-message p, #bottom p {color: #3f4042; font-size: 12px; font-family: Arial, Helvetica, sans-serif; }
  #header h1 {color: #ffffff !important; font-family: "Lucida Grande", sans-serif; font-size: 24px; margin-bottom: 0!important; padding-bottom: 0; }
  #header p {color: #ffffff !important; font-family: "Lucida Grande", "Lucida Sans", "Lucida Sans Unicode", sans-serif; font-size: 12px;  }
  .news {padding-bottom: 2px;}
  #content-4 h5 {
      font-size: 18px; 
      color: #444444 !important; 
      font-family: Arial, Helvetica, sans-serif; 
      margin: 0 0 0.8em 0; 
      }
  #content-4 h5 a {
    height: 80px !important;
    display: block;
    overflow: hidden! important;
    text-overflow: ellipsis! important;
    }
  #content-4 p { 
      font-size: 12px;
      color: #444444!important;
      font-family: "Lucida Grande","Lucida Sans","Lucida Sans Unicode",sans-serif;
      line-height: 1.5;
      overflow: hidden !important;
      text-overflow: ellipsis !important;
      max-height: 90px !important;
      margin: 0 0 0.8em 0; 
      }
   </style>
</head>
 
<body>

 
<table id="main" width="600" align="center" cellpadding="0" cellspacing="15" bgcolor="ffffff">
    <tr>
      <td>
        <table id="header" cellpadding="10" cellspacing="0" align="center" bgcolor="8fb3e9">
          <tr>
            <td width="570" align="center"  bgcolor="#d80a3e"><h1>Newsletter of Newsletters</h1></td>
          </tr>
          <tr>
            <td width="570" align="right" bgcolor="#d80a3e"></td>
          </tr>
        </table>
      </td>
    </tr>
 
    <tr>
      <td>
        <table id="content-3" cellpadding="0" cellspacing="0" align="center">
          <tr>
            <td width="50%" valign="top" style="padding:5px 25px 5px 5px;">
              <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0iv12vcOlIwmj9Irypthmhb6KH-FiNWadRDtwvn7rS4QF1fm2Gw&s" width="250" height="150"  />
            </td>
            <td width="50%" valign="top" style="padding:5px;">
                <img src="https://www.elderresearch.com/hs-fs/hubfs/BLOG_Data%20Engineering%20with%20Discipline.jpg?width=850&name=BLOG_Data%20Engineering%20with%20Discipline.jpg" width ="250" height="150" />
            </td>
          </tr>
        </table>
      </td>
    </tr>
    """

body_start="""<tr>
      <td>
        <table id="content-4" cellpadding="0" cellspacing="0" align="center">
          <tr>
            <td width="50%" valign="top" style="padding:5px;">"""

body_datascience=[]

for i in range(0,len(NewsLetter)):
  if NewsLetter["Source"][i]=='Data Science':
    t=Template("""
      <div class="news">
        <h5 class="newsLink"><a href={{link}}>{{title}}</a></h5>
        <p align="justify">{{summary}}..</p>
      </div>
        """)
    body_datascience.append(str(t.render(link=NewsLetter["Link"][i],title=NewsLetter["Title"][i],summary=NewsLetter["Summary"][i],
                                         source=NewsLetter["Source"][i])))
DS="".join(body_datascience)

body_seperater="""</td>   
            <td width="50%" valign="top" style="padding:5px;">"""


body_dataeng=[]
for i in range(0,len(NewsLetter)):
  if NewsLetter["Source"][i] == 'Data Engineering':
    t=Template("""
        <div class="news">
          <h5 class="newsLink"><a href={{link}}>{{title}}</a></h5>
          <p align="justify">{{summary}}..</p>
        </div>
          """)
    body_dataeng.append(str(t.render(link=NewsLetter["Link"][i],title=NewsLetter["Title"][i],summary=NewsLetter["Summary"][i],source=NewsLetter["Source"][i])))
DE="".join(body_dataeng)           

body_end= """</td>
          </tr>
        </table>
      </td>
    </tr>
  </table>"""
  

email_content=email_content+body_start+DS+body_seperater+DE+body_end+"""<table id="bottom" cellpadding="20" cellspacing="0" width="600" align="center">
    <tr>
      <td align="center">
        <p>Stay ahead & Be relevant</p>
      </td>
    </tr>
  </table><!-- top message -->
</td></tr></table><!-- wrapper -->
 
</body>
</html>"""

In [33]:
#!/usr/bin/env python

from smtplib import SMTP              # sending email
from email.mime.text import MIMEText  # constructing messages

from jinja2 import Environment        # Jinja2 templating

msg = email.message.Message()
msg['Subject'] = 'Newsletter of Newsletters'
 
#email_content="hi"
msg['From'] = "newsofnewsletter@gmail.com"
msg['To'] = "newsofnewsletter@gmail.com"
password="Student#1899!"
msg.add_header('Content-Type', 'text/html')
msg.set_payload((email_content).encode('ascii','ignore'))
 
s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()
 
# Login Credentials for sending the mail
s.login(msg['From'], password)
 
s.sendmail(msg['From'], msg["To"], msg.as_string())

{}